# The Battle of Boroughs in Munich

### There are 25 boroughs in Munich. In this notebook i want to compare the housing prices of the boroughs and find out the relationship between the prices and venues. At the end, i try to predict the housing prices of the boroughs in 2021.

### I mainly want to use the data of the venues from foursquare and the data of housing prices which i find in internet.

##### 1. get the venues data from foursquare

import the libraries

In [1]:
import numpy as np
import pandas as pd 
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

get the boroughs in Munich from Wikipedia

In [2]:
res=requests.get('https://de.wikipedia.org/wiki/Stadtbezirke_Münchens')
soup=BeautifulSoup(res.text,'html.parser')
tables=soup.select('table')
df_list=[]
for table in tables:
    df_list.append(pd.concat(pd.read_html(table.prettify())))
df=pd.concat(df_list)
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Ausländer (%),Dichte (Einw./km²),Einwohner,Fläche (km²),Nr.,Stadt- bezirks- nr.,Stadtbezirk,Stadtbezirksteile (Nr.)
0,260.0,6.716,21.126,315.0,1.0,NaN,Altstadt-Lehel,NaN
1,283.0,11.799,51.933,440.0,2.0,NaN,Ludwigsvorstadt-Isarvorstadt,NaN
2,256.0,12.060,51.834,430.0,3.0,NaN,Maxvorstadt,NaN
3,228.0,15.800,68.935,436.0,4.0,NaN,Schwabing-West,NaN
4,235.0,14.611,61.654,422.0,5.0,NaN,Au-Haidhausen,NaN


In [3]:
df=df.iloc[:25,[6,3]]


In [4]:
df.columns=['Boroughs','Area (km²)']


get the latitudes and longitudes of the boroughs from google map

In [5]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="Maxvorstadt"
country ="Germany"
loc = geolocator.geocode(city+','+ country)
print("latitude is :" ,loc.latitude,"\nlongtitude is:" ,loc.longitude)

latitude is : 48.1510916 
longtitude is: 11.5624179


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [6]:
from geopy.geocoders import Nominatim
geolocator=Nominatim()
lat=[]
lon=[]
for borough in df['Boroughs']:
    borough=borough
    country="Germany"
    loc=geolocator.geocode(borough+','+country)
    lat.append(loc.latitude)
    lon.append(loc.longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [7]:
df['Latitude']=lat
df['Longitude']=lon

In [54]:
df.head()

,Boroughs,Area (km²),Latitude,Longitude
0,Altstadt-Lehel,315.0,48.137828,11.574582
1,Ludwigsvorstadt-Isarvorstadt,440.0,48.130340,11.573366
2,Maxvorstadt,430.0,48.151092,11.562418
3,Schwabing-West,436.0,48.168271,11.569873
4,Au-Haidhausen,422.0,48.128753,11.590536


get venues from foursquare

In [9]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

get nearby venues

In [10]:
Client_ID="ZI4AGYQV250W0XUXT0IK015MFBHSRU2WP5DSJHTG1VYCGFRL"
Client_Secret="JBLAAX3KT3N1VWE0I41ZZGW5UXK3R2GKN5KQGSPXIHMK3WIF"
Version=20180605
limit=100
radius=500

In [11]:
def getNearbyVenues(names,latitudes,longitudes):
    venues_list=[]
    for name, lat, lon in zip(names,latitudes,longitudes):
        
        url='http://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(Client_ID,Client_Secret,Version,lat,lon,radius,limit)
        results=requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(name,lat,lon,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Borough','Borough Latitude','Borough Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

In [12]:
munich_venues=getNearbyVenues(df['Boroughs'],df['Latitude'],df['Longitude'])

In [58]:
munich_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt-Lehel,48.137828,11.574582,Marienplatz,48.137125,11.575483,Plaza
1,Altstadt-Lehel,48.137828,11.574582,Alois Dallmayr,48.138554,11.576750,Gourmet Shop
2,Altstadt-Lehel,48.137828,11.574582,Fischbrunnen,48.137211,11.576047,Fountain
3,Altstadt-Lehel,48.137828,11.574582,St. Peter,48.136530,11.575615,Church
4,Altstadt-Lehel,48.137828,11.574582,Augustiner Klosterwirt,48.138649,11.572527,German Restaurant


analyse the boroughs 

In [14]:
munich_onehot=pd.get_dummies(munich_venues[['Venue Category']],prefix="",prefix_sep="")
munich_onehot.head()

,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,...,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
munich_onehot['Borough']=munich_venues['Borough']
munich_onehot.head()

,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,...,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio,Borough
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Altstadt-Lehel
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Altstadt-Lehel
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Altstadt-Lehel
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Altstadt-Lehel
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Altstadt-Lehel


In [16]:
fixed_columns=[munich_onehot.columns[-1]]+list(munich_onehot.columns[:-1])
munich_onehot=munich_onehot[fixed_columns]
munich_onehot.head()

,Borough,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,...,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio
0,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
munich_grouped=munich_onehot.groupby('Borough').mean().reset_index()
munich_grouped.head()

,Borough,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,...,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio
0,Allach-Untermenzing,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0
1,Altstadt-Lehel,0.010000,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.010000,0.0,0.000000,0.01,0.0,0.0
2,Au-Haidhausen,0.025641,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.025641,0.0,0.025641,0.025641,0.0,0.000000,0.00,0.0,0.0
3,Berg am Laim,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0
4,Bogenhausen,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.166667,0.00,0.0,0.0


In [25]:
def most_common_venues(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]


In [19]:
num_top_venues=10
indicators=['st','nd','rd']
columns=['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} most common venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th most common venue'.format(ind+1))

borough_venues_sorted=pd.DataFrame(columns=columns)
borough_venues_sorted['Borough']=munich_grouped['Borough']


In [52]:
borough_venues_sorted.shape

(24, 11)

In [26]:
for i in np.arange(munich_grouped.shape[0]):
    borough_venues_sorted.iloc[i,1:]=most_common_venues(munich_grouped.iloc[i,:],10)

In [28]:
borough_venues_sorted

,Borough,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Allach-Untermenzing,Drugstore,Supermarket,Bakery,Sporting Goods Shop,Italian Restaurant,Automotive Shop,Food,Fish Market,Fast Food Restaurant,Farmers Market
1,Altstadt-Lehel,Café,Plaza,German Restaurant,Bavarian Restaurant,Hotel,Coffee Shop,Restaurant,Clothing Store,Church,Pizza Place
2,Au-Haidhausen,Italian Restaurant,Restaurant,Concert Hall,Coffee Shop,French Restaurant,Afghan Restaurant,Beach Bar,Beer Garden,Bistro,Cultural Center
3,Berg am Laim,Supermarket,Hotel,Bakery,Café,Drugstore,Gastropub,Metro Station,Gym,Grocery Store,Fast Food Restaurant
4,Bogenhausen,Italian Restaurant,Water Park,Bus Stop,Bank,Greek Restaurant,Yoga Studio,Drugstore,Fish Market,Fast Food Restaurant,Farmers Market
5,Feldmoching-Hasenbergl,Greek Restaurant,Motorcycle Shop,Yoga Studio,Food,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
6,Hadern,Bus Stop,Supermarket,Trattoria/Osteria,Sandwich Place,Sushi Restaurant,German Restaurant,Bakery,Ice Cream Shop,Dim Sum Restaurant,Cupcake Shop
7,Laim,Supermarket,Restaurant,Bank,Organic Grocery,Bakery,Drugstore,Mobile Phone Shop,Doner Restaurant,Sporting Goods Shop,Fried Chicken Joint
8,Ludwigsvorstadt-Isarvorstadt,Café,Bar,Cocktail Bar,Vietnamese Restaurant,Asian Restaurant,Italian Restaurant,Pizza Place,Ice Cream Shop,Afghan Restaurant,Gay Bar
9,Maxvorstadt,Café,Steakhouse,Coffee Shop,Vietnamese Restaurant,German Restaurant,Event Space,Martial Arts School,Peruvian Restaurant,Breakfast Spot,Israeli Restaurant


get housing price of the 25 boroughs in munich

In [ ]:
price_2020=pd.DataFrame({'€/m²':[7909.45,12108.30,7721.98,9439.62,7552.10,8739.03,7206.88,8155.08,8285.11,11066.33,10734.82,7651.63,7646.89,9991.34,7669.48,9031.81,7724.38,8880.87,9729.49,8156.29,8047.88,]})